# Credits
__This notebook has been adapted from__
- https://git.skewed.de/count0/graph-tool/-/wikis/installation-instructions#googles-colaboratory

# Installing graph-tool (https://graph-tool.skewed.de)

---


To import a library that's not in Colaboratory by default, you can use `!apt-get install`.

But since graph-tool is not in the official repository, we need to add it to the list.

In [ ]:
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

In [ ]:
#python3-cairo from Ubuntu's reposity is linked with a different python version; we need to improvise
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

# Repository of unweighted graphs: https://networks.skewed.de/?tags=Unweighted

In [ ]:
import numpy as np
import pandas as pd
import graph_tool.all as gt
from google.colab import files

from google.colab import drive

import os.path
from os import path

def export_all_unweightedGraphs_erodeCSV(model_names,download_file,store_on_gdrive,path_g_drive="unweightedGraphs/",prefix_to_weight=dict()):
   #Exports in ERODE CSV format all weighted graphs in the list model_names. 
   #The CSV file is either stored on google drive, or downloaded, as required by the parameters (Google drive has precedence on downloading)
   #If the google drive option is chosen, the files will be stored in folder weightedGraphs, or different paths provided in parameter path_g_drive
   #  This should be a relative path to a folder (ending with a slash). E.g., if you pass "weightedGraphs/", we will store files in it in MyDrive/weightedGraphs/


  print('Processing and exporting models')
  print("",model_names)

  #Prepare gdrive
  gdrive_path="/content/drive/MyDrive/"+path_g_drive
  if store_on_gdrive:
    drive.mount('/content/drive')
    if not path.exists(gdrive_path):
      os.mkdir(gdrive_path)
    if not path.exists(gdrive_path+"/csv/"):
      os.mkdir(gdrive_path+"/csv/")

  #Export required weighted graps
  for model_name in model_names:
    weight_name='weight'
    #for key in prefix_to_weight:
    #  if model_name.startswith(key):
    #    weight_name=prefix_to_weight[key]
    #    break
    export_weightedGraph_erodeCSV(model_name,download_file,store_on_gdrive,gdrive_path,weight_name)

  #Close gdrive
  if store_on_gdrive:
    drive.flush_and_unmount()

  print('All computations completed')

def export_unweightedGraph_erodeCSV(model_name,download_file,store_on_gdrive,gdrive_path,weight_name):
  #load model
  print("PROCESSING:",model_name,"...")

  g = gt.collection.ns[model_name]

  #if not(weight_name in g.edge_properties):
  #  print(" MODEL DROPPED BECAUSE NOT WEIGHTED")
  #  return
  add_rev= not g.is_directed()
    
  max_nodes=200
  n_nodes=g.num_vertices()
  if n_nodes > max_nodes:
    print(" MODEL DROPPED BECAUSE IT HAS MORE THAN",max_nodes,"NODES")
    return        
    
  #n_nodes=len(g.get_edges())
  #if add_rev:
  #      n_edges=n_edges*2
  #if n_edges > max_edges:
  #  print(" MODEL DROPPED BECAUSE IT HAS MORE THAN",max_edges,"EDGES")
  #  return        

  model_name = model_name.replace("/","_")
  model_name = model_name.replace(" ","_")
  model_name = model_name.replace("-","_")
  model_name = model_name.replace("(","_")
  model_name = model_name.replace(")","_")

  #store unweighted edges in a dataframe, and add weights (1)
  df = pd.DataFrame(data=g.get_edges(), columns=["source", "target"])
  df["source"]=df["source"]+1
  df["target"]=df["target"]+1
  
  df['weight']=np.ones(len(df))
  #print("BEWARE:",model_name,"DOES NOT HAVE WEIGHTS.\n WE SET ALL WEIGHTS TO ONE!")
  model_name=model_name+"_defaultWeights"


  #add_rev_str= str(add_rev)
  if add_rev:
      df_rev = df[["target","source",'weight']]
      df_rev.columns = ["source", "target",'weight']
      df=pd.concat([df,df_rev])
      df.reset_index(drop=True, inplace=True)
      add_rev=False
      

  #n_nodes=g.num_vertices()
  df0 = pd.DataFrame([[n_nodes,n_nodes,n_nodes]], columns=["source", "target",'weight'])
  df=pd.concat([df0, df])
  df.reset_index(drop=True, inplace=True)
  #print(df)

  dfB = pd.DataFrame(data=np.zeros(n_nodes), columns=["B"])

    
  #compute file name of csv file
  csv_file_name="csv/"+model_name+'A.csv'
  csv_file_nameB="csv/"+model_name+'B.csv'
  erode_file_name="import_"+model_name+'.ode'
    
  if store_on_gdrive:
    csv_file_name  =gdrive_path+"csv/"+model_name+'A.csv'
    csv_file_nameB =gdrive_path+"csv/"+model_name+'B.csv'
    erode_file_name=gdrive_path+"import_"+model_name+'.ode'
    
  

  print("  ","Graph processed, and data structure created")

  #create csv file and download if required
  df.to_csv( csv_file_name ,index=False,header=False)
  dfB.to_csv(csv_file_nameB,index=False,header=False)

  # Create ERODE file to import the CSV ones
  with open(erode_file_name, 'w') as fp:
    fp.write('begin model import_'+model_name+"\n")
    fp.write(' importAffineSystem(fileIn="csv/'+model_name+'A.csv", BFile="csv/'+model_name+'B.csv", ICFile="csv/'+model_name+'B.csv", addReverseEdges=false)\n')
    #fp.write(' write(fileOut="ODE/'+model_name+'._ode",format=ODE)\n')
    #fp.write(' reduceBE(reducedFile="BE/'+model_name+'_BE._ode", csvFile="BE.csv")\n')
    #fp.write(' reduceFE(reducedFile="FE/'+model_name+'_FE._ode", csvFile="FE.csv")\n')
    fp.write(' write(fileOut="EULER_ODE/'+model_name+'.ode",format=EULER)\n')
    #fp.write(' reduceRndFME(aggregationFunction=TIMES,reducedFile="RndFME_user/'+model_name+'_RndFME_user._ode", csvFile="RndTIMES_USER.csv,prePartition=USER")\n')
    fp.write('end model\n')
    
  
  if download_file and not store_on_gdrive:
    files.download(csv_file_name)
    files.download(csv_file_nameB)
    files.download(erode_file_name)

  print("  ","All files exported")

    
  
#Choose the models to consider
# Just https://networks.skewed.de/net/blumenau_drug
#model_names=["blumenau_drug"]
#model_names=["cattle"]
#model_names=["blumenau_drug","cattle","train_terrorists"]

#All weighted networks on 21/07/2022 from smallest number of nodes to largest#
# Taken from https://networks.skewed.de/?tags=Unweighted after clicking on 'Nodes'
model_names=[
    'sa_companies','florentine_families','high_tech_company','moreno_taro','november17','sp_baboons',
    'dutch_school','zebras','7th_graders','karate','dutch_criticism','montreal','ceo_club','elite',
    'macaque_neural','sp_kenyan_households','contiguous_usa','cs_department','dolphins','terrorists_911',
    'law_firm','sp_hospital','reality_mining','swingers','polbooks','adjnoun','fresh_webs','sp_hypertext',
    'football','football_tsevans','revolution','email_company','sp_high_school_new','foodweb_little_rock',
    'student_cooperation','jazz_collab','interactome_pdz','physician_trust','contact','malaria_genes',
    'sp_high_school','marvel_partnerships','kidnappings','facebook_friends','ugandan_village','sp_colocation',
    'ecoli_transcription','eu_airlines','celegans_metabolic','wiki_science','internet_top_pop','copenhagen',
    'yeast_transcription','uni_email','euroroad','faa_routes','crime','polblogs','collins_yeast','interactome_stelzl',
    'interactome_yeast','plant_pol_robertson','kegg_metabolic','board_directors','dnc','interactome_figeys',
    'us_air_traffic','urban_streets','interactome_vidal','power','facebook_organizations','fullerene_structures',
    'jung','celegans_interactomes','reactome','foursquare','jdk','route_views','software_dependencies',
    'sp_infectious','word_adjacency','dblp_cite','anybeat','chicago_road','google','genetic_multiplex',
    'marvel_universe','internet_as','word_assoc','cora','movielens_100k','caida_as','digg_reply','nematode_mammal',
    'linux','arxiv_citation','scotus_majority','topology','hiv_transmission','email_enron','inploid','pgp_strong',
    'paris_transportation','slashdot_threads','python_dependency','gnutella','lkml_reply','marker_cafe',
    'epinions_trust','twitter_15m','prosper','wiki_link_dyn','livemocha','ego_social','foursquare_friendships',
    'arxiv_authors','dbpedia_writer','lastfm_aminer','wordnet','dbtropes_feature','douban','dbpedia_starring',
    'github','dbpedia_recordlabel','dbpedia_producer','academia_edu','google_plus','flickr_aminer',
    'dbpedia_location','dbpedia_occupation','email_eu','dbpedia_genre','discogs_label','stanford_web',
    'notre_dame_web','corporate_directors','openstreetmap','lkml_thread','citeseer','amazon_copurchases',
    'dblp_coauthor_snap','bookcrossing','twitter','flickr_groups','visualizeus','dbpedia_country','petster',
    'stackoverflow','yahoo_ads','berkstan_web','myspace_aminer','citeulike','google_web','dbpedia_team',
    'bibsonomy','reuters','lastfm','wikitree','hyves','trec_web','as_skitter','trec','discogs_genre',
    'dblp_coauthor','wikipedia_growth','roadnet','discogs_affiliation','wiki_categories','baidu','twitter_events',
    'flickr_growth','wikipedia-en-talk','flixster','qa_user','mag_history_coauthor','mag_geology_coauthor',
    'wiki_talk','livejournal_aminer','us_patents','foursquare_global','dbpedia_all','livejournal','mislove_osn',
    'twitter_sample','dblp_simplices','bitcoin','dblp_author_paper','wikipedia_link','dbpedia_link','us_roads',
    'delicious','trackers','twitter_social','twitter_2009','soc_net_comms']

#Unfortunately, not all models classified as weighted have weights. We handle these cases setting deafult weight 1
#model_names=["hens"] # DOES NOT HAVE WEIGHTS! The default '1' will be set to all edges

# prefix_to_weight={'eu_procurements_alt/':'pctSingleBid','arxiv_collab/':'value','bag_of_words/':'count',
#                   'budapest_connectome/':'occurences','celegans_2019':'connectivity',
#                   'us_agencies/':'link_counts','plant_pol_vazquez':'count'}

#Set flags on where to store the obtained CSV files
download_file=False
store_on_gdrive=True
path_g_drive="weightedGraphs/"
export_all_unweightedGraphs_erodeCSV(model_names,download_file,store_on_gdrive,path_g_drive)

# If interested in drawing

In [ ]:
# from graph_tool.all import *
# g = collection.data["celegansneural"]
# state = minimize_nested_blockmodel_dl(g)
# state = minimize_nested_blockmodel_dl(g)
# state.draw()